# PyWren RISECamp, 2017

## Data Analytics with PyWren

In this section, we will use PyWren explore a dataset of Wikipedia records.

## 0. The Data

We've prepared an S3 bucket with 20GB of Wikipedia traffic statistics data obtained from http://aws.amazon.com/datasets/4182. To make the analysis more feasible for the short time you're here, we've shortened the dataset to three days worth of data (May 5 to May 7, 2009; roughly 20G and 329 million entries). 

Let's take a look into the bucket with our dataset. We'll print a few files from a few files from our bucket.

***Execute*** the code below to print out the names of the first 20 files.


In [ ]:
# These lines are only needed for the solutions.
import sys
sys.path.append("..")

# some libraries that are useful for this tutorial
from training import wikipedia_bucket, list_keys_with_prefix, read_from_s3

filenames = list_keys_with_prefix(wikipedia_bucket, "wikistats_20090505_restricted-01/")
for filename in filenames[:20]:
    print(filename)

There are 74 files (2 of which are intentionally left empty). Each file consists of a list of records. Let's go take a look into the first file. This may take a while.

***Execute*** the code below to print out 20 records from the file.

In [ ]:
# Read a file into memory, and split on newlines.
records = read_from_s3(wikipedia_bucket, "wikistats_20090505_restricted-01/part-00001").split("\n")

print("The total number of records in this file is {}, but here are the first 20".format(len(records)))
for i in range(20):
    print(records[i])

Each record has stats for a single Wikipedia page. The schema is:

`<date_time> <project_code> <page_name> <page_views> <page_size>`

- `<date_time>` specifies a date in YYYYMMDD-HHmmSS format (year, month, day, hour minute, second).
- `<project_code>` specifies the language the page is written in.
- `<page_title>` gives the page title.
- `<page_views>` gives the number of page views in the hour-long time slot starting at `<data_time>`. 
- `<page_size>` gives the size in bytes of the  page.

Now that we have a better understanding of the structure of our data, we can start running some interesting queries. 

Because the data are so large, it takes us quite a while to load just one file, as we saw in the previous exercise. You could imagine it would take way too long to process all of the data sequentially. Thankfully, we have [PyWren](https://www.youtube.com/watch?v=VSaDPc1Cs5U).

## 1. To the ~~Batmobile~~ Cloud!!!!!!

***Execute*** the code below to initialize an executor.


In [ ]:
# We need to load PyWren and create an executor instance
import pywren
pwex = pywren.default_executor()

## 2. Count
Let’s see how many records in total are in this data set.

***Exercise***: modify `count` to return the number of records in a given file.


In [ ]:
def count(filename):
    data = read_from_s3(wikipedia_bucket, filename)
    return (len(data.split("\n")) if data else 0)    

print ("invoking pywren jobs...")
futures = pwex.map(count, filenames)
print ("working...")
pywren.wait(futures)

result = sum([f.result() for f in futures])
print(result)

This should launch 73 PyWren tasks. After finishing the job, let's plot again to check the execution. This should look more interesting than the simple job before.

***Execute*** the code below to show a plot of the executions.

In [ ]:
from training import plot_pywren_execution
plot_pywren_execution(futures)

# 3. Visits for English Pages
Recall from above that the second field of a record is the `project code` For example, the project code `en` indicates an English page. Let’s calculate the view count on English pages for each date in our dataset.

***Exercise:*** modify `is_page_english` so that it return true if a given record corresponds to an English page.

In [ ]:
from itertools import groupby
from operator import itemgetter
from functools import reduce

def aggregate_count(key_value_list):
    
    def reduce_f(obj1, obj2):
        return(obj1[0], obj1[1] + obj2[1])
    
    counts = [reduce(reduce_f, group) for _, group 
          in groupby(sorted(key_value_list), key=itemgetter(0))]
    return counts

def english_page_count(filename):
    data = read_from_s3(wikipedia_bucket, filename)
    
    # return True if a record corresponds to an english page.
    def is_page_english(page):
        if len(page.split(" ")) >= 4 and page.split(" ")[1] == "en":
            return True
        return False
    
    # filter out the english pages
    en_pages = filter(is_page_english, data.split("\n"))
    
    # projection to create (date, pagecount) pairs
    def make_date_viewcount_pair(record):
        split_record = record.split()
        
        # the daate is the first entry of the record.
        # We only want the YYYYMMDD characters.
        date = split_record[0][:8]
        
        view_count = int(split_record[3])
        return (date, view_count)
    
    en_kvpair_list = [make_date_viewcount_pair(p) for p in en_pages]

    return aggregate_count(en_kvpair_list)
    
futures = pwex.map(english_page_count, filenames)
pywren.wait(futures)

results = [f.result() for f in futures]
en_page_counts_by_date = aggregate_count([x for y in results for x in y])
print(en_page_counts_by_date)